In [56]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

2. Keypoints using MP Holistic

In [57]:
mp_holistic_model = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 

In [58]:
def mediapipe_detection(frame , model):
    frame  = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB) # model expect RGB 
    frame.flags.writeable = False # save some memory while processing
    results = model.process(frame) # making prediction
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame , cv2.COLOR_RGB2BGR) # converting back to BGR
    return frame , results


In [1]:
def draw_landmarks(frame, results):
    # Draw face connections
    mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic_model.FACEMESH_TESSELATION)
    # Draw pose connections
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic_model.POSE_CONNECTIONS) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic_model.HAND_CONNECTIONS)
    # Draw right hand connections
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic_model.HAND_CONNECTIONS)


In [2]:
def draw_styled_landmarsks(frame, results):
    # Draw face connections
    mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic_model.FACEMESH_TESSELATION, 
                            #joint color , thickness , circle radius
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                            #lines color , thinkness
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic_model.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=2)
                            )  
    # Draw left hand connections
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic_model.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=2)
                            )   
    # Draw right hand connections  
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic_model.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4), 
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=2)
                            )  

Here is the explaination of what the following block of code is doing
-   we capture the frame using webcamp
-   we pass it to the holistic model to get keypoint
-   we pass tha image and the key points to function that draw these keypoint on the image

In [61]:
cap = cv2.VideoCapture(0)
#so here we are setting mpmodel
with mp_holistic_model.Holistic(min_detection_confidence=0.5 , min_tracking_confidence=0.5) as holistic :
    while cap.isOpened():
        ret , frame = cap.read()
        
        # making detection
        frame , results = mediapipe_detection(frame , model=holistic)
        # drawing landmarks
        draw_styled_landmarsks(frame , results)
        #showing to the screen
        cv2.imshow('openCV read' , frame)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

3. Extract Keypoint Values

In [62]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [63]:
# extracting the landmarks in one big array 
results_test = extract_keypoints(results=results)
# the number of landmarks we get for each frame
print(results_test.shape)